In [ ]:
import json

In [ ]:
with open("/mnt/disk6/Alfred/Rescoring/data/aishell2/data/withLM/train/data.json") as f:
    gen_data = json.load(f)

In [ ]:
with open("/mnt/disk6/Alfred/Rescoring/data/aishell2/raw/withLM/train/data.json") as f:
    data = json.load(f)

In [ ]:
print(type(data))

In [ ]:
data.keys()

In [ ]:
print(type(gen_data))

In [ ]:
count =0
s = []
for d in data['utts'].keys():
    if (len(data['utts'][d])) <= 1:
        count += 1
        s.append(d)

print(f'{count}')
for a in s:
    print(f"{a}\n")

In [ ]:
count =0
s = []
for d in gen_data:
    if (len(d['hyps'])) <= 1:
        count += 1
        s.append(d)

print(f'{count}')
for a in s:
    print(f"{a}\n")

In [ ]:
num_no50 = 0
for d in data:
    if (len(d['token']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['token'])}")
print(num_no50)

In [ ]:
num_no50 = 0
for d in data:
    if (len(d['pll']) != len(d['token'])):
        print(f"illegal:{d['name']}")
    if (len(d['pll']) != 50):
        num_no50 += 1
        print(f"{d['name']} is {len(d['pll'])}")
print(num_no50)